# Plot Source Time in Tract

 work with Weakly_2023_44
- use jupyter kernel LSST
- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2023/12/29
- update : 2023/12/29


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm 
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.dates as mdates
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle

plt.rcParams["figure.figsize"] = (16,10)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
# A repo which exists at the USDF, contains data from various instruments and
REPO = '/repo/embargo'

from lsst.daf.butler import Butler
from pprint import pprint

# Here we specify a default collection to use
my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553"
butler = Butler(REPO, collections=my_collection)
registry = butler.registry

In [ ]:
#import lsst.sphgeom
from  lsst.geom import SpherePoint, Angle

In [ ]:
Angle(3.1415).asDegrees()

In [ ]:
Angle(3.1415).asRadians()

In [ ]:
SpherePoint(Angle(3.1415),Angle(1.0))

In [ ]:
import astropy.coordinates as coord
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy import coordinates

In [ ]:
import pandas as pd

In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

## Config

### Galactic Plane

In [ ]:
gal_long = np.linspace(-180.,180,360)
gal_lat = np.zeros((360))

coordinates_galactic_planes = SkyCoord(l=gal_long*u.degree, b=gal_lat*u.degree, frame='galactic')
gp_radec = coordinates_galactic_planes.transform_to('icrs')  

gp_ra_toplot = coordinates.Angle(gp_radec.ra.degree*u.degree)
gp_ra_toplot = gp_ra_toplot.wrap_at(180*u.degree)

### Sources

In [ ]:
file = "sourceTable_202312.csv"

In [ ]:
suptitle = "Sources, Auxtel phot 2023"

## Skymap, Tracts and Patchs

In [ ]:
Auxtel2023_tracts = [ 5616,5634,5615,10644,383,8188,3863,455,452,5839,10643,382]

In [ ]:
skymap_auxtel ='latiss_v1'

In [ ]:
skymap = butler.get('skyMap', skymap=skymap_auxtel, collections=my_collection)

In [ ]:
with open('Latiss_tractspatchs.pickle', 'rb') as handle:
    tracts_patchs_dict = pickle.load(handle)

In [ ]:
tracts_patchs_dict

### Should try Tracts
- 5615 - OK , Not in our galaxy
- 5616 - Non, 1 nuit
- 5839 - Non, 1 nuit
- 5634 - OK - Galaxie Plane

In [ ]:
tract_selected = 5615
suptitle += f" tract {tract_selected}"

In [ ]:
tractInfo = skymap.generateTract(tract_selected)

In [ ]:
tractInfo.contains 

In [ ]:
def SourceInTract(row):
    #coord_dec
    #coord_ra
    #print(row)
    try:
        ra = row["ra"]
        dec = row["dec"]
        ra_rad = ra*np.pi/180.
        dec_rad = dec*np.pi/180.
        spherepoint = SpherePoint(Angle(ra_rad),Angle(dec_rad))
        flag = tractInfo.contains( spherepoint)
        return flag
    except Exception as err:
        print(f"Unexpected {err=}, {type(err)=}")
        return False
        
    

## Input

In [ ]:
df = pd.read_csv(file,index_col=0)

In [ ]:
list_of_columns = sorted(list(df.columns))

In [ ]:
#for col in list_of_columns:
#    print(col)

## Selection

In [ ]:
df.dropna(inplace=True)
selection1 = df["extendedness_flag"] == False
selection2 = df["detect_isPrimary"] == True
df = df[selection1 & selection2 ]

In [ ]:
df[["ra","dec"]]

In [ ]:
df["inptract"]=df[["ra","dec"]].apply(SourceInTract)

In [ ]:
all_flags = []
for index, row in df.iterrows():
    all_flags.append(SourceInTract(row))

In [ ]:
df["intract"] = all_flags

In [ ]:
df = df[df["intract"]==True]

## Magnitudes

In [ ]:
df["psfMag"] = df["psfFlux"].map(convert_fluxtomag)
df["gaussianMag"] = df["gaussianFlux"].map(convert_fluxtomag)
df["calibMag"] = df["calibFlux"].map(convert_fluxtomag)

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(14,4))

df['psfMag'].plot(kind="hist",bins=50,range=(5,30),facecolor="g",logy=True,grid=True,title="psfMag",ax=ax1)
df['gaussianMag'].plot(kind="hist",bins=50,range=(5,30),facecolor="r",logy=True,grid=True,title="gaussianMag",ax=ax2)
df['calibMag'].plot(kind="hist",bins=50,range=(5,30),facecolor="grey",logy=True,grid=True,title="calibMag",ax=ax3)
plt.suptitle(suptitle,y=1.0,fontsize=16)
plt.tight_layout()


In [ ]:
selection3  = df["psfMag"] < 20.
df = df[selection3]

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(14,4))
df['psfMag'].plot(kind="hist",bins=50,range=(5,30),facecolor="g",logy=True,grid=True,title="psfMag",ax=ax1)
df['gaussianMag'].plot(kind="hist",bins=50,range=(5,30),facecolor="r",logy=True,grid=True,title="gaussianMag",ax=ax2)
df['calibMag'].plot(kind="hist",bins=50,range=(5,30),facecolor="grey",logy=True,grid=True,title="calibMag",ax=ax3)
plt.suptitle(suptitle,y=1.0,fontsize=16)
plt.tight_layout()

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(10,4))
df.plot.scatter(x='psfMag',y='calibMag',marker="o",alpha=0.2,ax=ax1,grid=True)
ax1.set_xlim(10.,21.)
ax1.set_ylim(10.,21.)
ax1.set_aspect('equal')

df.plot.scatter(x='gaussianMag',y='calibMag',marker="o",alpha=0.2,ax=ax2,grid=True)
ax2.set_xlim(10.,21.)
ax2.set_ylim(10.,21.)
ax2.set_aspect('equal')

df.plot.scatter(x='psfMag',y='gaussianMag',marker="o",alpha=0.2,ax=ax3,grid=True)
ax3.set_xlim(10.,21.)
ax3.set_ylim(10.,21.)
ax3.set_aspect('equal')

plt.suptitle(suptitle,y=1.0,fontsize=16)
plt.tight_layout()

## Convert dateobs into datetime

In [ ]:
df["dateobs"]

In [ ]:
df["Time"] = pd.to_datetime(df['dateobs'])

In [ ]:
df.Time

In [ ]:
fig,(ax1,ax2) = plt.subplots(2,1)
df.plot(x="Time",y="coord_ra",marker='o',c="b",lw=0.0,ax=ax1,grid=True)
df.plot(x="Time",y="coord_dec",marker='o',c="g",lw=0.0,ax=ax2,grid=True)
plt.suptitle(suptitle,y=1.0,fontsize=16)
plt.tight_layout()

In [ ]:
fig,(ax1,ax2) = plt.subplots(2,1)
df.plot(x="Time",y="ra",marker='o',c="b",lw=0.0,ax=ax1,grid=True)
df.plot(x="Time",y="dec",marker='o',c="g",lw=0.0,ax=ax2,grid=True)
plt.suptitle(suptitle,y=1.0,fontsize=16)
plt.tight_layout()

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(16,5))
df_sel = df["coord_ra"]
df_sel.plot(kind="hist",x="coord_ra",bins=45,range=(0,360),facecolor="b",ax=ax1,grid=True,logy=True)
df_sel = df["coord_dec"]
df_sel.plot(kind="hist",x="coord_dec",bins=55,range=(-90,15),facecolor="g",ax=ax2,grid=True,logy=True)
plt.suptitle(suptitle,y=1.0,fontsize=16)
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,10))
cut1 = (df["ra"]> 200.) &  (df["ra"]<300.)
cut2 = (df["dec"]> -30) &  (df["dec"]<-10)
df_sel = df[cut1 & cut2]
df_sel = df_sel[["coord_ra","coord_dec"]]
df_sel.plot(kind="scatter",x="coord_ra",y="coord_dec",marker=".",c="r",ax=ax,grid=True)
plt.gca().set_aspect('equal')
plt.suptitle(suptitle,fontsize=16)
plt.tight_layout()

## Sky map

In [ ]:
ra = coord.Angle(df["coord_ra"].values, unit=u.degree)
dec = coord.Angle(df["coord_dec"].values, unit=u.degree)
#
ra  = ra.wrap_at(180*u.degree).radian
dec = dec.radian
#
df["RA"] = ra
df["DEC"] = dec

In [ ]:
fig = plt.figure()
#ax = fig.add_subplot(111, projection="aitoff")
ax = fig.add_subplot(111, projection="mollweide")
ax.scatter(gp_ra_toplot.radian, gp_radec.dec.radian,c="g",label="Galactic Plane",s=1)
df.plot.scatter(x='RA',y='DEC',c='ra',colormap='seismic',grid=True,ax=ax,figsize=(10,8),title='AUXTEL photometry 2023')
ax.legend()

In [ ]:
fig = plt.figure()
#ax = fig.add_subplot(111, projection="aitoff")
ax = fig.add_subplot(111, projection="mollweide")
ax.scatter(gp_ra_toplot.radian, gp_radec.dec.radian,c="g",label="Galactic Plane",s=1)
df.plot.scatter(x='RA',y='DEC',c='dec',colormap='seismic',grid=True,ax=ax,figsize=(10,8),title='AUXTEL photometry 2023')
ax.legend()

In [ ]:
df["RAdeg"] = df["RA"]*180/np.pi
df["DECdeg"] = df["DEC"]*180/np.pi

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,10))
cut1 = (df["ra"]> 200.) &  (df["ra"]<300.)
cut2 = (df["dec"]> -30) &  (df["dec"]<-10)
df_sel = df[cut1 & cut2]
df_sel = df_sel[["RAdeg","DECdeg"]]
df_sel.plot(kind="scatter",x="RAdeg",y="DECdeg",marker=".",c="r",ax=ax,grid=True)
plt.gca().set_aspect('equal')
plt.title(suptitle,fontsize=16)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,10))
cut1 = (df["RAdeg"]> -130.) &  (df["RAdeg"]<-115.)
cut2 = (df["dec"]> -30) &  (df["dec"]<-10)
df_sel = df[cut1 & cut2]
df_sel = df_sel[["RAdeg","DECdeg"]]
df_sel.plot(kind="scatter",x="RAdeg",y="DECdeg",marker=".",c="r",ax=ax,grid=True)
plt.gca().set_aspect('equal')
plt.title(suptitle,fontsize=16)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,10))
cut1 = (df["RAdeg"]> -95.) &  (df["RAdeg"]<-80.)
cut2 = (df["dec"]> -30) &  (df["dec"]<-10)
df_sel = df[cut1 & cut2]
df_sel = df_sel[["RAdeg","DECdeg"]]
df_sel.plot(kind="scatter",x="RAdeg",y="DECdeg",marker=".",c="r",ax=ax,grid=True)
plt.gca().set_aspect('equal')
plt.title(suptitle,fontsize=16)